In [13]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
tweets = pd.read_csv("../../tweets.csv", index_col=None)

In [12]:
len(sent_tokenize(tweets["full_text"][3000])[0].split(" "))

21

In [15]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [21]:
model.encode(sent_tokenize(tweets["full_text"][3000])[0])

array([ 2.65435372e-02, -4.90928674e-03, -1.41032087e-02,  4.54940973e-03,
        2.03673616e-02, -4.04983712e-03, -3.19448113e-02, -4.62085828e-02,
        3.32260765e-02,  2.18130765e-03, -3.85924689e-02,  1.62339639e-02,
        6.19630627e-02, -2.65714023e-02, -1.06855491e-02, -5.89409135e-02,
        1.16439909e-02,  2.19619926e-02,  3.44421640e-02, -1.00090504e-02,
        1.76108722e-02, -5.03648371e-02,  4.24702652e-02,  4.14306000e-02,
        4.36257944e-02,  4.61019081e-04, -4.67617810e-02,  1.82687994e-02,
       -3.34984213e-02, -3.07285711e-02,  2.26923469e-02, -7.85237551e-03,
       -2.45575905e-02,  1.77477747e-02,  1.53859639e-06, -5.60226664e-02,
        6.52004778e-03, -5.06085679e-02,  1.50390333e-02,  1.71867926e-02,
        1.75919048e-02,  6.45219460e-02, -4.67285663e-02, -3.62733155e-02,
        8.08718130e-02, -1.46646583e-02,  2.82631107e-02,  5.50633706e-02,
       -1.96894947e-02, -5.94634516e-03, -5.87824779e-03, -4.40546311e-03,
       -2.95719486e-02,  

In [17]:
sentences = ["This is an example sentence", "Each sentence is converted"]

embeddings = model.encode(sentences)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [20]:
embeddings.shape

(2, 768)